In [ ]:
import mlflow
import pandas as pd

uri = ...  # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)

#see all pandas columns
pd.set_option('display.max_columns', None)

# Last table

In [136]:
runs = mlflow.search_runs(
            experiment_ids=['333391697323445885'],
            output_format="pandas"
        )

params = ['status', 'params.embedding_dim', 'params.top_k', 'params.SAE_fusion_strategy', 'params.group_type', 'params.reconstruction_loss', 'params.contrastive_coef', 'params.user_set', 'params.group_set', 'params.topk_inference',  'params.normalize_users_embeddings',  'metrics.NDCG20/mean', 'metrics.NDCG20/min', 'metrics.CommonItemsNDCG20/mean', 'metrics.Popularity/mean', 'params.recommender_strategy', 'params.note']
df = runs[params]

filterr = (df['params.note'].str.contains('sizes'))
choose_deactivation = ((df['params.note'].str.contains('without')) & (df['params.SAE_fusion_strategy'] == 'common_features')) | (~(df['params.note'].str.contains('without')) & (df['params.SAE_fusion_strategy'] == 'average'))
# filterr3 = ()

normal = (df['params.note'] == 'sizes_L2_with_acts')
deactivate = (df['params.note'] == 'sizes_L2_without_acts')
normalized = choose_deactivation & (df['params.note'].str.contains('normalized')) & (df['params.note'].str.contains('L2'))
cosine = choose_deactivation & (df['params.note'].str.contains('Cosine')) & ~(df['params.note'].str.contains('normalized'))
contrastive = choose_deactivation & (df['params.note'].str.contains('constrastive')) & ~(df['params.note'].str.contains('normalized'))

df = df[filterr]

for col in ['params.embedding_dim', 'params.top_k']:
    df[col] = df[col].astype(dtype='int')
for col in ['metrics.NDCG20/min', 'metrics.NDCG20/min', 'metrics.Popularity/mean', 'metrics.CommonItemsNDCG20/mean']:
    df[col] = df[col].astype('float')
    
    
df_exp = pd.DataFrame()
for experiment in [normal, deactivate, normalized, cosine, contrastive]:
    new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
    df_exp = pd.concat([df_exp, new_df])
df = df_exp

print(f"Number of finished runs on valid set: {len(df)}")


renaming = {
    'params.group_type': 'Type',
    'params.embedding_dim': 'Dims',
    "params.top_k": 'TopK',
    "params.SAE_fusion_strategy": 'Agg',
    "params.reconstruction_loss": "$\\text{Loss}_R$",
    "params.topk_inference": "Act",
    "params.normalize_users_embeddings": "Norm",
    "params.contrastive_coef": "$\\lambda$",
    
    "metrics.CommonItemsNDCG20/mean": "$\\text{NDCG}_{com}$",
    "metrics.NDCG20/min": "$\\text{NDCG}_{min}$",
    "metrics.NDCG20/mean": "$\\text{NDCG}_{avg}$",
    'metrics.Popularity/mean': "Pop"
}

lambda_map = {
    '0.5': 1.0,
    '0.0': 0.0,
    '0.3': 0.5
}

df = df.rename(columns=renaming)[renaming.values()]
df['$\\lambda$'] = df['$\\lambda$'].apply(lambda x: lambda_map.get(x, x))
df['Agg'] = df['Agg'].apply(lambda x: "Avg" if x == 'average' else "ComF")
df['Type'] = df['Type'].apply(lambda x: "Similar" if x == 'sim' else "Random" if x == 'random' else 'Outlier')
df = df.sort_values('Type', kind = 'stable', ascending=False).round(3)

df.to_latex(
    'app_last.tex',
    escape=False,
    float_format='%.3f',
    index=False,
    column_format="llllllll|rrrr"
    )

df.head()

Number of finished runs on valid set: 324


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/2689642695.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/2689642695.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/2689642695.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/2689642695.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_val

,Type,Dims,TopK,Agg,$\text{Loss}_R$,Act,Norm,$\lambda$,$\text{NDCG}_{com}$,$\text{NDCG}_{min}$,$\text{NDCG}_{avg}$,Pop
1167,Similar,1024,32,Avg,L2,True,False,0.0,0.583,0.627,0.803,0.573
1169,Similar,1024,64,Avg,L2,True,False,0.0,0.582,0.628,0.802,0.567
1171,Similar,1024,128,Avg,L2,True,False,0.0,0.584,0.631,0.802,0.564
1173,Similar,2048,32,Avg,L2,True,False,0.0,0.591,0.637,0.806,0.572
1175,Similar,2048,64,Avg,L2,True,False,0.0,0.590,0.636,0.805,0.571


# Movielens table

In [2]:
runs = mlflow.search_runs(
            experiment_ids=['523100174176986081'],
            output_format="pandas"
        )

params = ['status', 'params.embedding_dim', 'params.top_k', 'params.SAE_fusion_strategy', 'params.group_type', 'params.reconstruction_loss', 'params.contrastive_coef', 'params.user_set', 'params.group_set', 'params.topk_inference',  'params.normalize_users_embeddings',  'metrics.NDCG20/mean', 'metrics.NDCG20/min', 'metrics.CommonItemsNDCG20/mean', 'metrics.Popularity/mean', 'params.recommender_strategy', 'params.note']
df = runs[params]

filterr = (df['params.note'].str.contains('sizes'))
choose_deactivation = ((df['params.note'].str.contains('without')) & (df['params.SAE_fusion_strategy'] == 'common_features')) | (~(df['params.note'].str.contains('without')) & (df['params.SAE_fusion_strategy'] == 'average'))
# filterr3 = ()

normal = (df['params.note'] == 'sizes_L2_with_acts')
deactivate = (df['params.note'] == 'sizes_L2_without_acts')
normalized = choose_deactivation & (df['params.note'].str.contains('normalized')) & (df['params.note'].str.contains('L2'))
cosine = choose_deactivation & (df['params.note'].str.contains('Cosine')) & ~(df['params.note'].str.contains('normalized'))
contrastive = choose_deactivation & (df['params.note'].str.contains('constrastive')) & ~(df['params.note'].str.contains('normalized'))

df = df[filterr]

for col in ['params.embedding_dim', 'params.top_k']:
    df[col] = df[col].astype(dtype='int')
for col in ['metrics.NDCG20/min', 'metrics.NDCG20/min', 'metrics.Popularity/mean', 'metrics.CommonItemsNDCG20/mean']:
    df[col] = df[col].astype('float')
    
    
df_exp = pd.DataFrame()
for experiment in [normal, deactivate, normalized, cosine, contrastive]:
    new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
    df_exp = pd.concat([df_exp, new_df])
df = df_exp

print(f"Number of finished runs on valid set: {len(df)}")


renaming = {
    'params.group_type': 'Type',
    'params.embedding_dim': 'Dims',
    "params.top_k": 'TopK',
    "params.SAE_fusion_strategy": 'Agg',
    "params.reconstruction_loss": "$\\text{Loss}_R$",
    "params.topk_inference": "Act",
    "params.normalize_users_embeddings": "Norm",
    "params.contrastive_coef": "$\\lambda$",
    
    "metrics.CommonItemsNDCG20/mean": "$\\text{NDCG}_{com}$",
    "metrics.NDCG20/min": "$\\text{NDCG}_{min}$",
    "metrics.NDCG20/mean": "$\\text{NDCG}_{avg}$",
    'metrics.Popularity/mean': "Pop"
}

lambda_map = {
    '0.5': 1.0,
    '0.0': 0.0,
    '0.3': 0.5
}


df = df.rename(columns=renaming)[renaming.values()]
df['$\\lambda$'] = df['$\\lambda$'].apply(lambda x: lambda_map.get(x, x))
df['Agg'] = df['Agg'].apply(lambda x: "Avg" if x == 'average' else "ComF")
df['Type'] = df['Type'].apply(lambda x: "Similar" if x == 'sim' else "Random" if x == 'random' else 'Outlier')
df = df.sort_values('Type', kind = 'stable', ascending=False).round(3)

df.to_latex(
    'app_movie.tex',
    escape=False,
    float_format='%.3f',
    index=False,
    column_format="llllllll|rrrr"
    )

df.head()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_68671/3602766177.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_68671/3602766177.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_68671/3602766177.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_68671/3602766177.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_val

Number of finished runs on valid set: 324


,Type,Dims,TopK,Agg,$\text{Loss}_R$,Act,Norm,$\lambda$,$\text{NDCG}_{com}$,$\text{NDCG}_{min}$,$\text{NDCG}_{avg}$,Pop
1492,Similar,1024,32,Avg,L2,True,False,0.0,0.643,0.555,0.701,0.503
1493,Similar,1024,64,Avg,L2,True,False,0.0,0.646,0.557,0.703,0.502
1494,Similar,1024,128,Avg,L2,True,False,0.0,0.648,0.559,0.704,0.500
1489,Similar,2048,32,Avg,L2,True,False,0.0,0.645,0.555,0.702,0.504
1491,Similar,2048,64,Avg,L2,True,False,0.0,0.646,0.557,0.702,0.500


In [117]:
runs = mlflow.search_runs(
            experiment_ids=['523100174176986081', '333391697323445885'],
            output_format="pandas"
        )


Number of finished runs on valid set: 81


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/707789077.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/707789077.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/707789077.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(['params.SAE_fusion_strategy', 'params.embedding_dim', 'params.top_k'])
/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_83451/707789077.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = df[experiment].sort_values(

,Type,Dims,TopK,Agg,$\text{Loss}_R$,Act,Norm,$\lambda$,$\text{NDCG}_{com}$,$\text{NDCG}_{min}$,$\text{NDCG}_{avg}$,Pop
621,Similar,2048,128,Avg,L2,True,False,0.5,0.647,0.558,0.704,0.498
622,Similar,4096,32,Avg,L2,False,False,0.5,0.550,0.601,0.782,0.646
625,Similar,4096,64,Avg,L2,False,False,0.5,0.569,0.617,0.792,0.625
626,Similar,4096,128,Avg,L2,True,False,0.5,0.647,0.557,0.703,0.497
890,Similar,1024,32,ComF,L2,False,False,0.5,0.575,0.623,0.799,0.564


In [ ]:

params = ['status', 'params.dataset' ,'params.embedding_dim', 'params.top_k', 'params.SAE_fusion_strategy', 'params.group_type', 'params.reconstruction_loss', 'params.contrastive_coef', 'params.user_set', 'params.group_set', 'params.topk_inference',  'params.normalize_users_embeddings',  'metrics.NDCG20/mean', 'metrics.NDCG20/min', 'metrics.CommonItemsNDCG20/mean', 'metrics.Popularity/mean', 'params.recommender_strategy', 'params.note']
df = runs[params]

filterr = (df['params.note'].str.contains('agg')) & (df['params.user_set'] == 'full') & (df['params.group_set'] == 'valid')

df = df[filterr]

for col in ['params.embedding_dim', 'params.top_k']:
    df[col] = df[col].astype(dtype='int')

for col in ['metrics.NDCG20/min', 'metrics.NDCG20/min', 'metrics.Popularity/mean', 'metrics.CommonItemsNDCG20/mean']:
    df[col] = df[col].astype('float')

print(f"Number of finished runs on valid set: {len(df)}")


renaming = {
    'params.dataset': "Dataset",
    'params.group_type': 'Type',
    'params.embedding_dim': 'Dims',
    "params.top_k": 'TopK',
    "params.SAE_fusion_strategy": 'Agg',
    "params.topk_inference": "Act",
    
    "metrics.CommonItemsNDCG20/mean": "$\\text{NDCG}_{com}$",
    "metrics.NDCG20/min": "$\\text{NDCG}_{min}$",
    "metrics.NDCG20/mean": "$\\text{NDCG}_{avg}$",
    'metrics.Popularity/mean': "Pop",
}

aggregations = {
    "common_features": "ComF",
    "average": "Avg",
    "wcom": "WCom",
    "topk": "TopK",
    "max": "Max",
}


df = df.rename(columns=renaming)[renaming.values()]
df['Agg'] = df['Agg'].apply(lambda x:aggregations[x])
df['Type'] = df['Type'].apply(lambda x: "Similar" if x == 'sim' else "Random" if x == 'random' else 'Outlier')
df = df.sort_values(["Dataset", 'Type', "Agg", "Dims", "TopK", "Act"], kind = 'stable', ascending=True).round(3)

df.to_latex(
    'agg.tex',
    escape=False,
    float_format='%.3f',
    index=False,
    column_format="llllll|rrrr"
    )

df.head(50)

Number of finished runs on valid set: 540


,Dataset,Type,Dims,TopK,Agg,Act,$\text{NDCG}_{com}$,$\text{NDCG}_{min}$,$\text{NDCG}_{avg}$,Pop
504,LastFM1k,Outlier,1024,32,Avg,False,0.430,0.468,0.718,0.625
526,LastFM1k,Outlier,1024,32,Avg,True,0.412,0.450,0.715,0.591
506,LastFM1k,Outlier,1024,64,Avg,False,0.398,0.443,0.706,0.569
529,LastFM1k,Outlier,1024,64,Avg,True,0.397,0.436,0.708,0.576
507,LastFM1k,Outlier,1024,128,Avg,False,0.389,0.431,0.700,0.553
532,LastFM1k,Outlier,1024,128,Avg,True,0.399,0.438,0.707,0.572
510,LastFM1k,Outlier,2048,32,Avg,False,0.447,0.477,0.723,0.666
534,LastFM1k,Outlier,2048,32,Avg,True,0.420,0.452,0.718,0.587
513,LastFM1k,Outlier,2048,64,Avg,False,0.440,0.479,0.720,0.621
536,LastFM1k,Outlier,2048,64,Avg,True,0.414,0.450,0.715,0.585
